# FuzzyChinese 應用及實作

為了更了解課程提到的 FuzzyChinese 套件，依據 [Github](https://github.com/znwang25/fuzzychinese) 進行實作及測試，並應用於現實場景。

In [1]:
# 安裝 fuzzychinese
!pip install fuzzychinese

In [27]:
# 載入套件
from fuzzychinese import FuzzyChineseMatch

fcm = FuzzyChineseMatch()

由說明文件我們可以得知此函數主要有兩個參數，分別是 `ngram_range` 及 `analyzer`，第一個就是採用最小 N-gram 到最大 N-gram，並全部納入辭庫；第二個則是作為相似度的依據：筆劃分析(stroke)，部首分析(radical)，和單字分析(char)。

為了進一步了解分析方法，我們可以調用函數出來比較。

In [1]:
from fuzzychinese import Radical, Stroke

In [2]:
stroke = Stroke()
radical = Radical()

In [3]:
stroke.get_stroke('我')

'㇒一㇚㇀㇂㇒㇔'

In [4]:
radical.get_radical('我')

'手戈'

這裡讓我們來找一個實例，最近我想做醫院叫號系統，當中每個醫院的科別可能有些許不同，也因此民眾在查詢時可能找不到完全匹配的字，抑或是打錯部分字詞。這裡我們嘗試把榮總診別抓出來，並作為 fuzzychinese 中的 target。

In [109]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
resp = requests.get('https://www6.vghtpe.gov.tw/reg/realTime.do?type=realtime')

In [8]:
soup = BeautifulSoup(resp.text)

In [60]:
department = soup.table.td.get_text().split('\n')
department = map(lambda x: re.sub(r'[^\u4E00-\u9FFF]+', '', x), department)
department = list(filter(lambda x: x, department))
department[:10]

['內科系',
 '一般內科',
 '一般體檢科',
 '職業醫學科',
 '臨床毒物科',
 '內科整合醫學門診',
 '家庭醫學科',
 '高齡醫學整合門診',
 '神經內科',
 '神經內科動作障礙特診']

In [68]:
fcm.fit(department)

FuzzyChineseMatch(analyzer=stroke, ngram_range=(3, 3))

In [118]:
def predict_result(test):
    return pd.DataFrame(zip(*fcm.transform([test]), *fcm.get_similarity_score()), columns = ['Predicted Words', 'Similarity Score'])

In [119]:
predict_result('一般体檢科')

,Predicted Words,Similarity Score
0,一般體檢科,0.763556
1,一般外科,0.605221
2,一般內科,0.586601


In [128]:
predict_result('一般科')

,Predicted Words,Similarity Score
0,一般外科,0.877347
1,一般內科,0.873212
2,一般體檢科,0.573610


In [131]:
predict_result('眼睛')

,Predicted Words,Similarity Score
0,眼科,0.726867
1,身心失眠,0.380710
2,輔具及功能重建門診,0.363313


In [127]:
predict_result('艮科')

,Predicted Words,Similarity Score
0,眼科,0.875055
1,喉科,0.305272
2,耳科,0.288272


In [126]:
predict_result('睡眠中止症')

,Predicted Words,Similarity Score
0,睡眠醫學中心,0.548074
1,睡眠障礙,0.500702
2,身心失眠,0.379093


In [132]:
predict_result('黃斑')

,Predicted Words,Similarity Score
0,青少年心理,0.373535
1,基因體檢測門診,0.335666
2,自費心理諮詢,0.325997


仔細觀察後發現，匹配成效基本上不差，但如果需要深入到病灶名稱，就會有匹配不上的問題。另外 眼科 改用 艮科 查詢，也可以看到很高的匹配程度，基本是拜 fuzzychinese 所賜。然而，其他表現不錯的原因卻與此套件無關，主要貢獻仍然是 N-Gram 匹配前後字串所致。因此若是此套件應用在打字輸入的錯誤比對上，會顯得較沒有優勢。然而，如果是針對手寫字體辨識後產出的文字檔，就能一定程度的改善手寫勘誤問題。